# Tp2 - Análise de desempenho

In [4]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import time

## Criando o banco de dados

Para executar os códigos neste notebook, é necessário possuir o banco de dados relacional do IMDB no mesmo diretório desse notebook. Caso você ainda não possua o banco de dados, você deve seguir esses passos:

1. Instale o imdb-sqlite seguindo as instruções em: https://pypi.org/project/imdb-sqlite/
2. Rode o imdb-sqlite na mesma pasta desse notebook.

Caso você já possua o banco de dados, basta movê-lo para o mesmo diretório deste notebook.


## Abrindo o banco de dados

In [3]:
conn = sqlite3.connect('imdb.db')

## Análise de desempenho

Como utilizamos um banco de dados grande, podemos ver o impacto no tempo de execução de diferentes formas de se realizar a mesma  consulta. Nesta seção vamos analisar e comparar algumas consultas.

### 1º Consulta: Seleção antes da junção Vs Junção antes da seleção

Primeiramente faremos a Seleção antes da junção:

In [18]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df1 = pd.read_sql_query(
"""
SELECT *
FROM ratings NATURAL JOIN(
    SELECT *
    FROM titles
    WHERE original_title="O Auto da Compadecida"
)

""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df1

Tempo gasto: 0.02031707763671875


,title_id,rating,votes,type,primary_title,original_title,is_adult,premiered,ended,runtime_minutes,genres
0,tt0197139,9.0,2332,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
1,tt0271383,8.6,9099,movie,A Dog's Will,O Auto da Compadecida,0,2000,NaN,104.0,"Adventure,Comedy"


Com a junção "antes" da seleção:

In [17]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df2 = pd.read_sql_query(
"""
SELECT *
FROM ratings NATURAL JOIN titles
WHERE original_title="O Auto da Compadecida"
   
""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df2

Tempo gasto: 0.015127420425415039


,title_id,rating,votes,type,primary_title,original_title,is_adult,premiered,ended,runtime_minutes,genres
0,tt0197139,9.0,2332,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
1,tt0271383,8.6,9099,movie,A Dog's Will,O Auto da Compadecida,0,2000,NaN,104.0,"Adventure,Comedy"


### 2º Consulta: Junção Vs Produto Cartesiano

Junção antes:

In [23]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df1 = pd.read_sql_query(
"""
SELECT *
FROM ratings NATURAL JOIN(
    SELECT *
    FROM titles
    WHERE original_title="O Auto da Compadecida"
)

""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df1

Tempo gasto: 0.021650314331054688


,title_id,rating,votes,type,primary_title,original_title,is_adult,premiered,ended,runtime_minutes,genres
0,tt0197139,9.0,2332,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
1,tt0271383,8.6,9099,movie,A Dog's Will,O Auto da Compadecida,0,2000,NaN,104.0,"Adventure,Comedy"


Produto Cartesiano:

In [26]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df2 = pd.read_sql_query(
"""
SELECT *
FROM ratings as R, titles as T
WHERE original_title="O Auto da Compadecida" and R.title_id=T.title_id


""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df2

Tempo gasto: 0.014823436737060547


,title_id,rating,votes,title_id,type,primary_title,original_title,is_adult,premiered,ended,runtime_minutes,genres
0,tt0197139,9.0,2332,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
1,tt0271383,8.6,9099,tt0271383,movie,A Dog's Will,O Auto da Compadecida,0,2000,NaN,104.0,"Adventure,Comedy"


Mostrando o tempo gasto por um produto cartesiano:

In [27]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df3 = pd.read_sql_query(
"""
SELECT *

FROM ratings as R, titles as T
WHERE original_title="O Auto da Compadecida" 


""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df3

Tempo gasto: 8.952706575393677


,title_id,rating,votes,title_id,type,primary_title,original_title,is_adult,premiered,ended,runtime_minutes,genres
0,tt0000001,5.8,1513,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
1,tt0000002,6.3,184,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
2,tt0000003,6.6,1160,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
3,tt0000004,6.3,113,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
4,tt0000005,6.2,1865,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
5,tt0000006,5.4,100,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
6,tt0000007,5.5,606,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
7,tt0000008,5.6,1614,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
8,tt0000009,5.5,80,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"
9,tt0000010,6.9,5387,tt0197139,tvMiniSeries,O Auto da Compadecida,O Auto da Compadecida,0,1999,1999.0,NaN,"Comedy,Drama,Romance"


### Consulta 3: IN Vs EXISTS Vs INNER JOIN

In:

In [31]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df1 = pd.read_sql_query(
"""
SELECT rating, votes
FROM ratings as R 
WHERE R.title_id IN(
    SELECT T.title_id
    FROM titles as T
    WHERE T.original_title="O Auto da Compadecida"
)

""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df1

Tempo gasto: 0.0031423568725585938


,rating,votes
0,9.0,2332
1,8.6,9099


Exists:

In [41]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df2 = pd.read_sql_query(
"""
SELECT rating, votes
FROM ratings as R 
WHERE EXISTS (
    SELECT T.title_id
    FROM titles as T
    WHERE T.original_title="O Auto da Compadecida" and T.title_id=R.title_id
)

""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df2

Tempo gasto: 1.3026010990142822


,rating,votes
0,9.0,2332
1,8.6,9099


Inner Join:

In [42]:
inicio = time.time()

# Consulta todos os filmes que tem o titulo original e pertencem a região Br
df3 = pd.read_sql_query(
"""
SELECT R.rating, R.votes
FROM ratings as R INNER JOIN titles as T
WHERE T.original_title="O Auto da Compadecida" and T.title_id=R.title_id


""", conn)


fim = time.time()
print('Tempo gasto: {}'.format(fim - inicio))

df3

Tempo gasto: 0.0023767948150634766


,rating,votes
0,9.0,2332
1,8.6,9099
